In [25]:
from mycode.common import *

构造数据

In [26]:
metadata, tables = fetch_data_from_sqlite(path='./mycode/1k_data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
result_table = tables["BookLoan"]
x_table = ["Book", "Library", "Student", "Enrollment", "Submission"]
x_key = ['book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
print(f"{len(result_table)=}")
for i, x_table_name in enumerate(x_table):
    result_table = pd.merge(result_table, tables[x_table_name], on=x_key[i], how=x_how[i])
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
#result_table

len(result_table)=1000
1000 1000 1000 1014 1193 23


In [27]:
# result_table = result_table.sample(n=1000)
result_table

,loan_id,book_id,student_id,loan_date,return_date,title,author,isbn,library_id,name_x,...,date_of_birth,major,enrollment_id,course_id,semester,grade_x,submission_id,assignment_id,submission_date,grade_y
0,4,4,260,2023-10-26,2025-09-03,Late rich.,Dr. Terri David,978-1-168-13965-8,4,"Williams, Moore and Kim Library",...,1994-07-14,734,309,967,Spring 2020,D,19,798,2022-04-01 16:19:43,C
1,6,6,944,2023-12-16,2025-03-22,Attack fish international.,Zachary Lawrence,978-1-02-166559-1,6,Ramirez-Randall Library,...,2003-01-18,935,500,425,Spring 2021,A,99,485,2022-01-31 04:46:38,B
2,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,79,876,2022-02-09 23:36:49,I
3,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,856,213,2022-08-04 10:27:59,F
4,7,7,931,2023-10-17,2025-04-29,Class person.,Jason Holland,978-1-59433-218-0,7,Lang LLC Library,...,1994-10-06,140,789,657,Spring 2019,A,972,668,2022-12-28 21:53:58,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,998,998,211,2023-11-09,2025-06-30,Gun land drive.,Sherry Hurst,978-1-4658-0306-1,998,Johnson Ltd Library,...,2006-01-23,812,372,334,Summer 2022,C,799,229,2022-04-06 19:02:31,I
1189,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,298,942,Summer 2019,None,106,497,2022-11-21 04:00:27,B
1190,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,298,942,Summer 2019,None,315,166,2022-04-05 07:47:40,I
1191,999,999,126,2024-04-16,2024-11-24,Common election.,Caleb Steele,978-1-06-971080-2,999,"Mclean, Riley and Olson Library",...,1999-10-20,775,822,419,Winter 2020,B,106,497,2022-11-21 04:00:27,B


检查metadata

In [28]:
result_table.to_csv("test_1k_label.csv", index=False)
dataset_csv = "test_1k_label.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-10-14 20:24:09.284 | INFO     | sdgx.data_models.metadata:from_dataloader:294 - Inspecting metadata...
2024-10-14 20:24:09.860 | INFO     | sdgx.data_models.metadata:update_primary_key:503 - Primary Key updated: set().


{'author',
 'date_of_birth',
 'email',
 'grade_x',
 'grade_y',
 'isbn',
 'loan_date',
 'location',
 'name_x',
 'name_y',
 'return_date',
 'semester',
 'submission_date',
 'title'}

In [29]:
loan_metadata.datetime_columns

{'date_of_birth', 'loan_date', 'return_date', 'submission_date'}

In [30]:
loan_metadata.column_list.__len__()

23

In [31]:
loan_metadata.datetime_format = {
    key: "%Y-%m-%d" for key in loan_metadata.datetime_columns if key != "submission_date" 
}
loan_metadata.datetime_format["submission_date"] = "%Y-%m-%d %H:%M:%S"
loan_metadata.datetime_format

{'return_date': '%Y-%m-%d',
 'date_of_birth': '%Y-%m-%d',
 'loan_date': '%Y-%m-%d',
 'submission_date': '%Y-%m-%d %H:%M:%S'}

In [32]:
loan_metadata.discrete_columns = set([
    key for key in loan_metadata.discrete_columns if key not in loan_metadata.datetime_columns
])
# loan_metadata.label_columns = loan_metadata.discrete_columns
#loan_metadata.label_columns.add("grade_x")
#loan_metadata.label_columns.add("grade_y")
print(f"{loan_metadata.int_columns=}\n{loan_metadata.float_columns=}\n{loan_metadata.const_columns=}\n{loan_metadata.bool_columns=}\n{loan_metadata.discrete_columns=}")

loan_metadata.int_columns={'course_id', 'enrollment_id', 'submission_id', 'loan_id', 'student_id', 'major', 'library_id', 'assignment_id', 'book_id'}
loan_metadata.float_columns=set()
loan_metadata.const_columns=set()
loan_metadata.bool_columns=set()
loan_metadata.discrete_columns={'name_x', 'author', 'title', 'grade_y', 'isbn', 'semester', 'grade_x', 'email', 'location', 'name_y'}


In [33]:
loan_metadata.column_encoder = {
    key: "label" for key in loan_metadata.discrete_columns
}

训练

In [34]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    metadata=loan_metadata,
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()

# synthesizer.save("test_model_100k")

2024-10-14 20:24:09.959 | INFO     | sdgx.synthesizer:__init__:106 - Using data processors: ['nonvaluetransformer', 'outliertransformer', 'emailgenerator', 'chnpiigenerator', 'intvalueformatter', 'datetimeformatter', 'constvaluetransformer', 'emptytransformer', 'columnordertransformer']
2024-10-14 20:24:09.960 | INFO     | sdgx.synthesizer:fit:296 - Fitting data processors...
2024-10-14 20:24:09.987 | INFO     | sdgx.data_processors.transformers.nan:fit:71 - NonValueTransformer Fitted.
2024-10-14 20:24:09.988 | INFO     | sdgx.data_processors.transformers.nan:fit:87 - NonValueTransformer get int columns: {'course_id', 'enrollment_id', 'loan_id', 'student_id', 'submission_id', 'library_id', 'major', 'assignment_id', 'book_id'}.
2024-10-14 20:24:09.988 | INFO     | sdgx.data_processors.transformers.nan:fit:96 - NonValueTransformer get float columns: set().
2024-10-14 20:24:09.989 | INFO     | sdgx.data_processors.transformers.nan:fit:101 - NonValueTransformer get column list from metadat

Preparing data:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-14 20:24:10.144 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column loan_id...
2024-10-14 20:24:10.322 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column book_id...
2024-10-14 20:24:10.495 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column student_id...
2024-10-14 20:24:10.654 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column loan_date...
2024-10-14 20:24:10.817 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:127 - Fitting continuous column return_date...
2024-10-14 20:24:10.992 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:122 - Fitting discrete column title...
2024-10-14 20:24:10.997 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:122 - Fitting discrete column author.

Transforming data:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-14 20:24:17.430 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:253 - Sampling data.
2024-10-14 20:24:17.431 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:258 - Initialize Generator.
2024-10-14 20:24:17.433 | INFO     | sdgx.models.ml.single_table.ctgan:fit:223 - CTGAN prefit finished, start CTGAN training.
2024-10-14 20:24:17.434 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:276 - Fit using data_size:1193, data_dim: 117.
2024-10-14 20:24:17.436 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:302 - Starting model training, epochs: 1


[ColumnTransformInfo(column_name='loan_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=7, activation_fn='softmax')], output_dimensions=8), ColumnTransformInfo(column_name='book_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7), ColumnTransformInfo(column_name='student_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=6, activation_fn='softmax')], output_dimensions=7), ColumnTransformInfo(column_name='loan_date', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=7, activation_fn='softmax')], output_dimensions=8), Column

Fitting batches:   0%|          | 0/2 [00:00<?, ?it/s]

2024-10-14 20:24:17.542 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:383 - Epoch 1, Loss G:  0.0474, Loss D:  0.0212, Time:  0.1057
2024-10-14 20:24:17.543 | INFO     | sdgx.models.ml.single_table.ctgan:fit:225 - CTGAN training finished.
2024-10-14 20:24:17.543 | INFO     | sdgx.synthesizer:fit:333 - Model fit... Finished


In [35]:
data_res = synthesizer.sample(1000)

2024-10-14 20:24:17.552 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

Sampling batches:   0%|          | 0/3 [00:00<?, ?it/s]

Inverse transforming:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-14 20:24:17.834 | INFO     | sdgx.data_processors.transformers.nan:reverse_convert:155 - Data reverse-converted by NonValueTransformer.
2024-10-14 20:24:17.836 | INFO     | sdgx.data_processors.transformers.outlier:reverse_convert:120 - Data reverse-converted by OutlierTransformer (No Action).
2024-10-14 20:24:18.040 | INFO     | sdgx.data_processors.formatters.int:reverse_convert:64 - Data reverse-converted by IntValueFormatter.
2024-10-14 20:24:18.040 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:165 - Data reverse-converting by DatetimeFormatter...
2024-10-14 20:24:18.041 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:167 - parameters : ['return_date', 'submission_date', 'date_of_birth', 'loan_date'], {'return_date': '%Y-%m-%d', 'date_of_birth': '%Y-%m-%d', 'loan_date': '%Y-%m-%d', 'submission_date': '%Y-%m-%d %H:%M:%S'}
2024-10-14 20:24:18.057 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:173 - Data reve

In [36]:
data_res.to_csv("test_1k_simulate.csv", index=False)